# 3D-PhysGenRD 기반 Solid Grain 설계
이 노트북은 3D SDF 기반 형상 표현, latent diffusion, 물리 서러게이트를 사용해
정방향(Forward) 성능 예측과 역설계(Reverse Design)를 수행합니다.


In [ ]:
import torch
import matplotlib.pyplot as plt

from physgenrd import (
    CurveEncoder,
    DiffusionConfig,
    GrainConfig,
    LatentDiffusion,
    NeuralSDFField,
    PhysicsSurrogate,
    PositionalEncoding,
    TrainingConfig,
    forward_performance,
    make_grid,
    reverse_design,
    set_seed,
)


In [ ]:
cfg = GrainConfig()
diff_cfg = DiffusionConfig()
train_cfg = TrainingConfig()
set_seed(cfg.seed)

time_grid = torch.arange(0.0, cfg.t_end + cfg.dt, cfg.dt)
pc_target = torch.ones_like(time_grid) * 6.0e6
pc_target[time_grid > 2.2] *= torch.exp(-0.8 * (time_grid[time_grid > 2.2] - 2.2))


In [ ]:
# Forward: latent 샘플로부터 성능 예측
_, _, _, coords_flat = make_grid(cfg)
coords_flat = coords_flat.to(cfg.device)

encoder = PositionalEncoding().to(cfg.device)
curve_encoder = CurveEncoder().to(cfg.device)
cond = curve_encoder(pc_target.to(cfg.device).unsqueeze(0).unsqueeze(0)).squeeze(0)

sdf_field = NeuralSDFField(
    enc_dim=encoder(coords_flat[:1]).shape[-1],
    latent_dim=diff_cfg.latent_dim,
    cond_dim=cond.shape[-1],
).to(cfg.device)
surrogate = PhysicsSurrogate(cfg.grid_size).to(cfg.device)
diffusion = LatentDiffusion(diff_cfg, cond_dim=cond.shape[-1]).to(cfg.device)

z = diffusion.sample(cond)
forward_out = forward_performance(
    sdf_field, encoder, surrogate, coords_flat, cfg, z, cond
)

plt.figure(figsize=(8, 4))
plt.plot(time_grid.cpu().numpy(), pc_target.cpu().numpy() / 1e6, 'k--', label='Target')
plt.plot(time_grid.cpu().numpy(), forward_out['Pc'].cpu().numpy() / 1e6, 'r-', label='Predicted')
plt.xlabel('Time [s]')
plt.ylabel('Pressure [MPa]')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Reverse: 목표 압력 곡선에 맞는 3D 그레인 역설계
reverse_out = reverse_design(pc_target.to(cfg.device), cfg, diff_cfg, train_cfg)

plt.figure(figsize=(8, 4))
plt.plot(time_grid.cpu().numpy(), pc_target.cpu().numpy() / 1e6, 'k--', label='Target')
plt.plot(time_grid.cpu().numpy(), reverse_out['Pc'].cpu().numpy() / 1e6, 'r-', label='Achieved')
plt.xlabel('Time [s]')
plt.ylabel('Pressure [MPa]')
plt.legend()
plt.tight_layout()
plt.show()


## 참고
- `physgenrd.py` 모듈은 SDF 기반 형상 표현과 물리 서러게이트를 사용합니다.
- `reverse_design`은 목표 압력 곡선과 로딩 제약, 에이코날 잔차를 함께 최적화합니다.
